In [1]:
import mesa
from mesa.time import SimultaneousActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from mesa.visualization import Slider
from mesa.visualization.modules import CanvasGrid, ChartModule, TextElement
from mesa.visualization.ModularVisualization import ModularServer

from sklearn.metrics import mean_squared_error, mean_absolute_error
import pandas as pd
import numpy as np
import random
import math
import json

In [2]:
cities = pd.read_csv("data/cities.csv")
demographic = pd.read_csv("data/demographic.csv")
turnout = pd.read_csv("data/turnout.csv")

In [3]:
total_population_by_city = demographic.groupby("city")["population"].sum()
city_names = [row["city"] for _, row in cities.iterrows()]

In [ ]:
# Add parameters that you can adjust to see how it influences voter turnout
model_params = {
    "cost_peak": Slider("Cost Peak", 0.4, 0, 1, 0.01),
    "cost_variance": Slider("Cost Variance", 0.5, 0, 1, 0.01),
    "majority_influence": Slider("Majority Influence", 0.3, 0, 1, 0.01)
}

In [ ]:
class Agent(mesa.Agent):
    def __init__(self, id, model, city, sex, age_group, income, education):
        super().__init__(id, model)
        self.id = id
        self.city = city
        self.sex = sex
        self.age_group = age_group
        self.income = income
        self.education = education
        self.state = "Registered" # Registered, WillVote
        self.voting_neighbors = 0 # voting neighbors / number of neighbors
    
    def compute_benefit(self):
        with open("data/weights.json") as file:
            weight = json.load(file)
            age = weight["age"].get(self.age_group, 1.0)
            sex = weight["sex"].get(self.sex, 1.0)
            income = weight["income"].get(self.income, 1.0)
            education = weight["education"].get(self.education, 1.0)

            if self.voting_neighbors > 0.5: # majority of neighbors intend to vote
                social_utility = self.model.majority_influence
            else:
                social_utility = 0
            expressive_benefit = (age + sex + income + education) / 4
            benefit = social_utility + expressive_benefit
            # print(self.id, age, sex, income, education, social_utility, benefit)
            return benefit

    def step(self):
        self.move()

        # gets surrounding 8 cells and gets the proportion of those that intend to vote
        neighbors = [n for pos in self.model.grid.get_neighborhood(
                self.pos, include_center=False, moore=True) 
             for n in self.model.grid.get_cell_list_contents([pos])]
        neighbor_count = len(neighbors)
        self.voting_neighbors = sum(1 for a in neighbors if a.state == "WillVote") / neighbor_count if neighbor_count != 0 else 0

        benefit = self.compute_benefit()
        # cost = random.uniform(0, 1)
        cost = random.gauss(self.model.cost_peak, self.model.cost_variance)
        
        R = benefit - cost
        
        if R > 0: # the benefits outweigh the costs
            self.state = "WillVote"
        else:
            self.state = "Registered"

    
    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=False, # von neumann neighborhood: left, up, down, right only. no diagonals
            include_center=True # can stay in place
        )
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

In [6]:
def calculate_weight(row):
    city = row["city"]
    population = row["population"]

    demographic_proportion = population / total_population_by_city[city]
    registration_rate = turnout[turnout["city"] == city]["registered"].values[0] / total_population_by_city[city]
    return demographic_proportion * registration_rate

def normalize(val, min_val, max_val):
    return (val - min_val) / (max_val - min_val)

In [ ]:
total_registered = turnout["registered"].sum()
simulated_registered_by_city = {}
spawn_radius = 5

class Model(mesa.Model):    
    def __init__(self, N=1000, cost_peak=0.4, cost_variance=0.5, majority_influence=0.3):
        self.num_agents = N
        width = int(math.sqrt(self.num_agents) * 2)
        height = width
        self.grid = MultiGrid(width, height, True)
        self.schedule = SimultaneousActivation(self)
        self.running = True
        self.datacollector = DataCollector(
            model_reporters={
                "Percentage of registered agents that intend to vote": lambda m: 100 * sum(1 for a in m.schedule.agents if a.state == "WillVote") / len(m.schedule.agents),
                **{
                city: (lambda c, city=city: c.vote_intent_by_city()[city]) 
                for city in city_names
                }
            }
        )
        # Set model parameters
        self.cost_peak = cost_peak
        self.cost_variance = cost_variance
        self.majority_influence = majority_influence

        # Generate grid coordinates for each city based on real-world coordinates
        min_lon, max_lon = cities["longitude"].min(), cities["longitude"].max()
        min_lat, max_lat = cities["latitude"].min(), cities["latitude"].max()

        city_centers = {}
        for _, row in cities.iterrows():
            norm_x = normalize(row["longitude"], min_lon, max_lon)
            norm_y = normalize(row["latitude"], min_lat, max_lat)

            x = int(norm_x * (self.grid.width - 1))
            y = int((1 - norm_y) * (self.grid.height - 1))

            city_centers[row["city"]] = (x, y)
        

        # Create N agents with demographic data proportional to the population
        agents = demographic.copy()
        agents["weight"] = agents.apply(lambda row: calculate_weight(row), axis=1)
        
        agents["normalized_weight"] = agents["weight"] / agents["weight"].sum()
        agents["allocated"] = (agents["normalized_weight"] * self.num_agents).astype(int)
        remaining = self.num_agents - agents["allocated"].sum()
        remainders = (agents["normalized_weight"] * self.num_agents) - agents["allocated"]
        agents.loc[remainders.nlargest(remaining).index, "allocated"] += 1
        # print(agents)
        i = 0
        for _, row in agents.iterrows():
            city = row["city"]
            age_group = row["age_group"]
            sex = row["sex"]
            income = row["income"]
            education = row["education"]
            
            alloc = int(row["allocated"])
            if city not in simulated_registered_by_city:
                simulated_registered_by_city[city] = alloc
            else:
                simulated_registered_by_city[city] += alloc
            
            for __ in range(alloc):
                agent = Agent(i, self, city, sex, age_group, income, education)
                self.schedule.add(agent)
                dx = self.random.randint(-spawn_radius, spawn_radius)
                dy = self.random.randint(-spawn_radius, spawn_radius)
                x = min(max(city_centers[city][0] + dx, 0), self.grid.width - 1)
                y = min(max(city_centers[city][1] + dy, 0), self.grid.height - 1)
                self.grid.place_agent(agent, (x,y))
                i += 1
    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()
    
    def vote_intent_by_city(self):
        vote_counts = {city: 0 for city in city_names}
        registered_counts = {city: 0 for city in city_names}

        for agent in self.schedule.agents:
            registered_counts[agent.city] += 1
            if agent.state == "WillVote":
                    vote_counts[agent.city] += 1

        vote_percentages = {}
        for city in city_names:
            total = registered_counts[city]
            vote_percentages[city] = (vote_counts[city] / total) if total > 0 else 0

        return vote_percentages


In [8]:
time_duration = 20
model = Model()
for t in range(time_duration):
    # print(f"Step {i+1}")
    model.step()

scale = total_registered / model.num_agents
# print(scale)


simulated_voted_count = {}
for agent in model.schedule.agents:
    if agent.city not in simulated_voted_count:
        simulated_voted_count[agent.city] = 0
    if agent.state == "WillVote":
        simulated_voted_count[agent.city] += 1

df_voted = turnout.set_index("city", inplace=False)["voted"].to_dict()
df_registered = turnout.set_index("city", inplace=False)["registered"].to_dict()

common_cities = set(df_voted.keys()) & set(simulated_voted_count.keys()) # cities in common with actual voting dataset and simulated

actual_voted = [df_voted[city] for city in common_cities]
actual_registered = [df_registered[city] for city in common_cities]
actual_turnout = [df_voted[city] / df_registered[city] for city in common_cities]

simulated_voted = [simulated_voted_count[city] * scale for city in common_cities]
simulated_registered = [simulated_registered_by_city[city] * scale for city in common_cities]
simulated_turnout = [simulated_voted_count[city] / simulated_registered_by_city[city] for city in common_cities]

comparison_df = pd.DataFrame({
    "City": list(common_cities),
    "Registered Voters": actual_registered,
    "Simulated Registered Voters": simulated_registered,
    
    "Actual Voters": actual_voted,
    "Simulated Voters": simulated_voted,

    "Actual Turnout": actual_turnout,
    "Simulated Turnout": simulated_turnout
})

comparison_df.to_csv("output/results.csv")

print(comparison_df.sort_values("City").reset_index(drop=True))

# Evaluation Metrics
rmse = np.sqrt(mean_squared_error(actual_turnout, simulated_turnout))
mae = mean_absolute_error(actual_turnout, simulated_turnout)
print(f"Root Mean Squared Error: {rmse: .2f}")
print(f"Mean Absolute Error: {mae: .2f}")

c:\Users\beaux\anaconda3\envs\consunji_beaux\lib\site-packages\mesa\agent.py:52: FutureWarning: The Mesa Model class was not initialized. In the future, you need to explicitly initialize the Model by calling super().__init__() on initialization.
  self.model.register_agent(self)


           City  Registered Voters  Simulated Registered Voters  \
0      caloocan             700279                   256282.635   
1     las piñas             291074                   300216.801   
2        makati             458362                   600433.602   
3       malabon             258115                   607755.963   
4   mandaluyong             232492                   380762.772   
5        manila            1133042                   439341.660   
6      marikina             260749                   373440.411   
7    muntinlupa             311750                   373440.411   
8       navotas             150693                   512565.270   
9     parañaque             346078                   329506.245   
10        pasay             276579                   483275.826   
11        pasig             457370                   395407.494   
12      pateros              39273                   373440.411   
13  quezon city            1403895                   300216.80

In [9]:
def agent_portrayal(a):
    city_color = {
        "manila": "#e6194b",
        "quezon city": "#3cb44b",
        "caloocan": "#ffe119",
        "makati": "#4363d8",
        "marikina": "#f58231",
        "parañaque": "#911eb4",
        "pasig": "#46f0f0",
        "valenzuela": "#f032e6",
        "pasay": "#bcf60c",
        "las piñas": "#fabebe",
        "mandaluyong": "#008080",
        "muntinlupa": "#e6beff",
        "malabon": "#9a6324",
        "navotas": "#fffac8",
        "san juan": "#800000",
        "taguig": "#aaffc3",
        "pateros": "#808000"
    }
    portrayal = {
        "Filled": "true",
        "r": 1,
        "w": 1,
        "h": 1,
        "Color": city_color[a.city],
        "Layer": 0
    }
    portrayal["Shape"] = "rect" if a.state == "WillVote" else "circle"
    
    return portrayal
class Legend(TextElement):
    def render(self, model):
        return """
        <style>
            .legend-grid {
                display: grid;
                grid-template-columns: repeat(3, auto);
                gap: 4px 12px;
                font-size: 12px;
                line-height: 1.2;
            }
            .legend-item {
                display: flex;
                align-items: center;
                gap: 4px;
            }
            .color-box {
                width: 12px;
                height: 12px;
                display: inline-block;
            }
        </style>
        <div><b>City Color Legend:</b></div>
        <div class="legend-grid">
            <div class="legend-item"><span class="color-box" style="background:#e6194b;"></span>Manila</div>
            <div class="legend-item"><span class="color-box" style="background:#3cb44b;"></span>Quezon City</div>
            <div class="legend-item"><span class="color-box" style="background:#ffe119;"></span>Caloocan</div>
            <div class="legend-item"><span class="color-box" style="background:#4363d8;"></span>Makati</div>
            <div class="legend-item"><span class="color-box" style="background:#f58231;"></span>Marikina</div>
            <div class="legend-item"><span class="color-box" style="background:#911eb4;"></span>Parañaque</div>
            <div class="legend-item"><span class="color-box" style="background:#46f0f0;"></span>Pasig</div>
            <div class="legend-item"><span class="color-box" style="background:#f032e6;"></span>Valenzuela</div>
            <div class="legend-item"><span class="color-box" style="background:#bcf60c;"></span>Pasay</div>
            <div class="legend-item"><span class="color-box" style="background:#fabebe;"></span>Las Piñas</div>
            <div class="legend-item"><span class="color-box" style="background:#008080;"></span>Mandaluyong</div>
            <div class="legend-item"><span class="color-box" style="background:#e6beff;"></span>Muntinlupa</div>
            <div class="legend-item"><span class="color-box" style="background:#9a6324;"></span>Malabon</div>
            <div class="legend-item"><span class="color-box" style="background:#fffac8;"></span>Navotas</div>
            <div class="legend-item"><span class="color-box" style="background:#800000;"></span>San Juan</div>
            <div class="legend-item"><span class="color-box" style="background:#aaffc3;"></span>Taguig</div>
            <div class="legend-item"><span class="color-box" style="background:#808000;"></span>Pateros</div>
        </div>
        """

legend = Legend()
# print(model.datacollector.get_model_vars_dataframe())

grid = CanvasGrid(agent_portrayal, model.grid.width, model.grid.height, 800, 800)
# TODO: Label axes of the chart -- Not possible by default with MESA

total_chart = ChartModule([{
    "Label": "Percentage of registered agents that intend to vote",
    "Color": "black"
}], data_collector_name="datacollector")

city_chart = ChartModule([
    {"Label": "manila", "Color": "#e6194b"},
    {"Label": "quezon city", "Color": "#3cb44b"},
    {"Label": "caloocan", "Color": "#ffe119"},
    {"Label": "makati", "Color": "#4363d8"},
    {"Label": "marikina", "Color": "#f58231"},
    {"Label": "parañaque", "Color": "#911eb4"},
    {"Label": "pasig", "Color": "#46f0f0"},
    {"Label": "valenzuela", "Color": "#f032e6"},
    {"Label": "pasay", "Color": "#bcf60c"},
    {"Label": "las piñas", "Color": "#fabebe"},
    {"Label": "mandaluyong", "Color": "#008080"},
    {"Label": "muntinlupa", "Color": "#e6beff"},
    {"Label": "malabon", "Color": "#9a6324"},
    {"Label": "navotas", "Color": "#fffac8"},
    {"Label": "san juan", "Color": "#800000"},
    {"Label": "taguig", "Color": "#aaffc3"},
    {"Label": "pateros", "Color": "#808000"}
], data_collector_name="datacollector")


In [10]:
server = ModularServer(
    Model,
    [legend, grid, total_chart, city_chart],
    "NCR Agent-based Voter Turnout Model",
    model_params
)
server.port = 4000

c:\Users\beaux\anaconda3\envs\consunji_beaux\lib\site-packages\mesa\agent.py:52: FutureWarning: The Mesa Model class was not initialized. In the future, you need to explicitly initialize the Model by calling super().__init__() on initialization.
  self.model.register_agent(self)


In [ ]:
server.launch()

Interface starting at http://127.0.0.1:4000


RuntimeError: This event loop is already running

Socket opened!
{"type":"reset"}


c:\Users\beaux\anaconda3\envs\consunji_beaux\lib\site-packages\mesa\agent.py:52: FutureWarning: The Mesa Model class was not initialized. In the future, you need to explicitly initialize the Model by calling super().__init__() on initialization.
  self.model.register_agent(self)


{"type":"get_step","step":1}
{"type":"get_step","step":2}
{"type":"get_step","step":3}
{"type":"get_step","step":4}
{"type":"get_step","step":5}
{"type":"get_step","step":6}
{"type":"get_step","step":7}
{"type":"get_step","step":8}
{"type":"get_step","step":9}
{"type":"get_step","step":10}
{"type":"get_step","step":11}
{"type":"get_step","step":12}
{"type":"get_step","step":13}
{"type":"get_step","step":14}
{"type":"get_step","step":15}
{"type":"get_step","step":16}
{"type":"get_step","step":17}
{"type":"get_step","step":18}
{"type":"reset"}


c:\Users\beaux\anaconda3\envs\consunji_beaux\lib\site-packages\mesa\agent.py:52: FutureWarning: The Mesa Model class was not initialized. In the future, you need to explicitly initialize the Model by calling super().__init__() on initialization.
  self.model.register_agent(self)


{"type":"submit_params","param":"cost_variance","value":0.52}
{"type":"submit_params","param":"cost_variance","value":0.53}
{"type":"submit_params","param":"cost_variance","value":0.54}
{"type":"submit_params","param":"cost_variance","value":0.55}
{"type":"submit_params","param":"cost_variance","value":0.57}
{"type":"submit_params","param":"cost_variance","value":0.58}
{"type":"submit_params","param":"cost_variance","value":0.6}
{"type":"submit_params","param":"cost_variance","value":0.61}
{"type":"submit_params","param":"cost_variance","value":0.63}
{"type":"submit_params","param":"cost_variance","value":0.64}
{"type":"submit_params","param":"cost_variance","value":0.65}
{"type":"submit_params","param":"cost_variance","value":0.66}
{"type":"submit_params","param":"cost_variance","value":0.67}
{"type":"submit_params","param":"cost_variance","value":0.68}
{"type":"submit_params","param":"cost_variance","value":0.69}
{"type":"submit_params","param":"cost_variance","value":0.7}
{"type":"s

c:\Users\beaux\anaconda3\envs\consunji_beaux\lib\site-packages\mesa\agent.py:52: FutureWarning: The Mesa Model class was not initialized. In the future, you need to explicitly initialize the Model by calling super().__init__() on initialization.
  self.model.register_agent(self)


{"type":"get_step","step":1}
{"type":"get_step","step":2}
{"type":"get_step","step":3}
{"type":"get_step","step":4}
{"type":"get_step","step":5}
{"type":"get_step","step":6}
{"type":"get_step","step":7}
{"type":"get_step","step":8}
{"type":"get_step","step":9}
{"type":"get_step","step":10}
{"type":"get_step","step":11}
{"type":"get_step","step":12}
{"type":"get_step","step":13}
{"type":"get_step","step":14}
{"type":"get_step","step":15}
{"type":"get_step","step":16}
{"type":"get_step","step":17}
{"type":"get_step","step":18}
{"type":"get_step","step":19}
{"type":"get_step","step":20}
{"type":"get_step","step":21}
{"type":"get_step","step":22}
{"type":"get_step","step":23}
{"type":"get_step","step":24}
{"type":"get_step","step":25}
{"type":"get_step","step":26}
{"type":"get_step","step":27}
{"type":"get_step","step":28}
{"type":"get_step","step":29}
{"type":"get_step","step":30}
{"type":"get_step","step":31}
{"type":"get_step","step":32}
{"type":"get_step","step":33}
{"type":"get_step",